In [9]:
import torch
from torch.utils.data import DataLoader
from models.transformer import Transformer, create_look_ahead_mask
from dataset_extracted import ExtractedFeatureDataset
import sentencepiece as spm
import transforms
from evaluate import Captioner
import os

In [10]:
flag = 'test'

In [11]:
test_feature_path = '../MVAD/I3D_rgb_kinetics/{}'.format(flag)
with open('../MVAD/{}_fine'.format(flag)) as f:
    files = f.readlines()
    feature_files = list(map(lambda file: os.path.join(test_feature_path, str.strip(file) + '.npy'), files))

In [12]:
test_corpus_file = '../MVAD/corpus_M-VAD_{}.txt'.format(flag)
tokenizer_file = 'tokenizer.model'
model_weight_file = '../checkpoint/20190814011228/36'
inp_max_seq_length = 50
tar_max_seq_length = 50
tar_vocab_size = 5000
d_model = 1024
num_heads = 8
num_layers = 6
encoder_num_layers = 3
decoder_num_layers = 4
dff = 2048
dropout = 0.1
max_seq_length = 80  # For positional encoding

In [13]:
sp = spm.SentencePieceProcessor()
sp.Load(tokenizer_file)

True

In [14]:
feature_transform = transforms.Compose([
    transforms.FeaturePadding(inp_max_seq_length)
])
caption_transform = transforms.Compose([
    transforms.CaptionPadding(tar_max_seq_length, sp.PieceToId('<PAD>'))
])

In [15]:
dataset = ExtractedFeatureDataset(None, test_corpus_file, inp_max_seq_length, tar_max_seq_length, sp, feature_transform=feature_transform, caption_transform=caption_transform, feature_files=feature_files)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [16]:
model = Transformer(tar_vocab_size, d_model, num_heads, encoder_num_layers, decoder_num_layers, dff, dropout, max_seq_length)
state_dict = torch.load(model_weight_file, map_location='cpu')
model.load_state_dict(state_dict['model_state_dict'], strict=True)
model.eval()

Transformer(
  (pe): PositionalEncoder()
  (encoder): TransformerEncoder(
    (encoder_layers): ModuleList(
      (encoder_layer1): TransformerEncoderLayer(
        (mha): MultiheadAttention(
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (ff): FeedForward(
          (linear1): Linear(in_features=1024, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1)
        )
        (an1): AddNorm(
          (dropout): Dropout(p=0.1)
          (layernorm): LayerNorm(torch.Size([1024]), eps=1e-05, elementwise_affine=True)
        )
        (an2): AddNorm(
          (dropout): Dropout(p=0.1)
          (layernorm): LayerNorm(torch.Size([1024]), eps=1e-05, elementwise_affine=True)
        )
      )
      (encoder_layer2): TransformerEncoderLayer(
        (mha): MultiheadAttention(
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )

In [17]:
captioner = Captioner(model, sp, tar_max_seq_length)
captioner.caption_video_from_dataloader(dataloader, count=10)

video: ('THE_BOUNTY_HUNTER_DVS79.avi',)
caption origin: He jiggles the locked door.
caption predict: SOMEONE'surther S title force full sp mask lurches.

video: ('UGLY_TRUTH_DVS253.avi',)
caption origin: She claps a hand over her cat's mouth, then continues to ogle her neighbor.
caption predict: She after stops, adjusts her chin cover un frameed pin unup dancers hits has in point control panel.

video: ('PUBLIC_ENEMIES_DVS460.avi',)
caption origin: SOMEONE and SOMEONE make their own escape.
caption predict: He glancesper somed file his mother's bra single file on fans

video: ('THE_BOUNTY_HUNTER_DVS40.avi',)
caption origin: She leaves. SOMEONE grins at the mirror.
caption predict: SOMEONE eyesum cut out, peeks belowying for hook til.

video: ('FRIENDS_WITH_BENEFITS_DVS20.avi',)
caption origin: In a hired car, SOMEONE uses her cell.
caption predict: Now at the wing S of an embo train his sadly.

video: ('INSIDE_MAN_DVS243.avi',)
caption origin: Outside, snipers keep their guns trained o

In [9]:
dataiter = iter(dataloader)

In [16]:
for sample in dataiter:
    sample[0].size()

TypeError: batch must contain tensors, numbers, dicts or lists; found <class 'NoneType'>